# KNN(k nearest neighbors)
## KNN可以做分類或回歸，為監督式學習
### 1. 設定k值
### 2. 計算距離公式找出k個最相近的特徵
### 3. 分類: k個特徵投票、回歸: 平均k個特徵
<img src="https://ww2.mathworks.cn/matlabcentral/mlc-downloads/downloads/03faee64-e85e-4ea0-a2b4-e5964949e2d1/d99b9a4d-618c-45f0-86d1-388bdf852c1d/images/screenshot.gif">

### 蒐集資料
python MLGame.py -i ml_play_template.py -f 200 -r arkanoid NORMAL 3

In [19]:
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

file = open("./log/P2(1).pickle", "rb")
data = pickle.load(file)
file.close()

type(data['ml_2P'])

dict

In [20]:
game_info = data['ml_2P']['scene_info']
game_command = data['ml_2P']['command']
print(game_info)
print(game_command)

[{'frame': 0, 'status': 'GAME_ALIVE', 'ball': (98, 415), 'ball_speed': (0, 0), 'platform_1P': (80, 420), 'platform_2P': (80, 50), 'blocker': (40, 240)}, {'frame': 1, 'status': 'GAME_ALIVE', 'ball': (98, 415), 'ball_speed': (0, 0), 'platform_1P': (80, 420), 'platform_2P': (80, 50), 'blocker': (45, 240)}, {'frame': 2, 'status': 'GAME_ALIVE', 'ball': (98, 415), 'ball_speed': (0, 0), 'platform_1P': (80, 420), 'platform_2P': (80, 50), 'blocker': (50, 240)}, {'frame': 3, 'status': 'GAME_ALIVE', 'ball': (98, 415), 'ball_speed': (0, 0), 'platform_1P': (80, 420), 'platform_2P': (80, 50), 'blocker': (55, 240)}, {'frame': 4, 'status': 'GAME_ALIVE', 'ball': (98, 415), 'ball_speed': (0, 0), 'platform_1P': (80, 420), 'platform_2P': (80, 50), 'blocker': (60, 240)}, {'frame': 5, 'status': 'GAME_ALIVE', 'ball': (98, 415), 'ball_speed': (0, 0), 'platform_1P': (80, 420), 'platform_2P': (80, 50), 'blocker': (65, 240)}, {'frame': 6, 'status': 'GAME_ALIVE', 'ball': (98, 415), 'ball_speed': (0, 0), 'platform

In [21]:
for i in range (2,77):
    path = "./log/P2(" + str(i) + ").pickle"
    file = open(path,"rb")
    data = pickle.load(file)
    game_info = game_info + data["ml_2P"]["scene_info"]
    game_command = game_command + data["ml_2P"]["command"]
    file.close
    
print(len(game_info))
print(len(game_command))

g = game_info[1]

119769
119769


### 特徵整理

In [22]:
feature = np.array([g["ball"][0], g["ball"][1], g["ball_speed"][0], g["ball_speed"][1], g["platform_2P"][0]+15,80])

print(feature)
print(game_command[1])
if game_command[1] == "NONE":
	game_command[1] = 0
elif game_command[1] == "MOVE_LEFT":
	game_command[1] = 1
else:
	game_command[1] = 2

[ 98 415   0   0  95  80]
None


In [23]:
for i in range(2, len(game_info) - 1):
    g = game_info[i]
    g_last = game_info[i-1]
    ball_x=g["ball"][0]
    ball_y=g["ball"][1]
    ball_vx=g['ball'][0] - g_last['ball'][0]
    ball_vy=g['ball'][1] - g_last['ball'][1]
    des_x = 80
    #if des_vy > 0:
    if ball_vy < 0:                                             #ball move up
                    if ball_vx < 0:                                         #ball move left
                        des_x = ball_x - (ball_y - 80)
                    else:                                                       #ball move right   
                        des_x = ball_x + (ball_y - 80)
    else:                                                           #ball move down  
                    if ball_vx < 0:                                         #ball move left
                        des_x = ball_x - (680 - (ball_y - 80))
                    else:                                                       #ball move right
                        des_x = ball_x + (680 - (ball_y - 80))
       
    while des_x > 200 or des_x < 0:
        if des_x > 200:
                des_x = 200 - (des_x - 200)
        else: 
                des_x = -des_x
    feature = np.vstack((feature, [g['ball'][0], g['ball'][1],ball_vx, ball_vy,g['platform_2P'][0]+15,des_x]))
    if game_command[i] == "NONE": game_command[i] = 0
    elif game_command[i] == "MOVE_LEFT": game_command[i] = 1
    else: game_command[i] = 2 
        
answer = np.array(game_command[1:-1])
print(feature)
print(feature.shape)
print(answer)
print(answer.shape)

[[ 98 415   0   0  95  80]
 [ 98 415   0   0  95  43]
 [ 98 415   0   0  95  43]
 ...
 [189 404  18  18 140 145]
 [195 422   6  18 140 133]
 [190 440  -5  18 140 130]]
(119767, 6)
[2 2 2 ... 0 0 1]
(119767,)


### KNN官方文件
https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html
### 交叉驗證
https://chih-sheng-huang821.medium.com/%E4%BA%A4%E5%8F%89%E9%A9%97%E8%AD%89-cross-validation-cv-3b2c714b18db

In [24]:
x_train, x_test, y_train, y_test = train_test_split(feature, answer, test_size=0.3, random_state = 0) #0.3 0
#參數區間
forest = RandomForestClassifier(n_estimators=195, max_depth=175) #195 175
forest = forest.fit(x_train, y_train)
predicted = forest.predict(x_test)
file = open("mine_model_random2.pickle", "wb")
pickle.dump(forest, file)
file.close()
print("Accuracy score (validation): {0:.3f}".format(forest.score(x_test, y_test)))
print("Confusion Matrix for Raandom Forests:")
print(confusion_matrix(y_test, predicted))
print()
print("Classification Report for Random Forests")
print(classification_report(y_test, predicted))

Accuracy score (validation): 0.996
Confusion Matrix for Raandom Forests:
[[16815     9    56]
 [   36  7451     0]
 [   42     0 11522]]

Classification Report for Random Forests
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     16880
           1       1.00      1.00      1.00      7487
           2       1.00      1.00      1.00     11564

    accuracy                           1.00     35931
   macro avg       1.00      1.00      1.00     35931
weighted avg       1.00      1.00      1.00     35931



### f1-score
https://medium.com/nlp-tsupei/precision-recall-f1-score%E7%B0%A1%E5%96%AE%E4%BB%8B%E7%B4%B9-f87baa82a47

In [131]:
#最佳參數
#print(grid.best_params_)
#預測結果
#print(grid_predictions)
#混淆矩陣
#print(confusion_matrix(y_test, grid_predictions))
#分類結果
#print(classification_report(y_test, grid_predictions))

### 執行遊戲
python MLGame.py -i knn.py -f 50 arkanoid NORMAL 3